In [97]:
# B9XP3X
# THANAKORN PHUTTHARAKSA

# Exercise 1 (8 points = 6 + 2)
# The cyclotomic polynomials Φ(n) can be calculated as follows. ("Φ" is the greek capital letter "phi".)
# The n’th polynomial, Φ(n) is xn −1 divided by all Φ(k) polynomials for which k is a proper divisor
# of n. For example, the proper divisors of n = 10 are k = 1, 2, 5, so Φ(10) := x10 − 1
# Φ(1)Φ(2)Φ(5). Note
# that Φ(1) can be calculated by itself, since n = 1 has no proper divisors: Φ(1) = x − 1.
# a) Create the cyclotomic polynomials Φ(n) from n = 1 to 20 using the above method. The type
# of the results must be polynomials, not Sage symbolic expressions. (Don’t use the built-in
# function cyclotomic_polynomial(n) other than for checking.)
# b) What is the degree of the polynomial Φ(n), which well-known function describes it? (Hint: the
# greek letter helps.) Make a guess, and then check the guess using Sage from n = 1 to 20.

Pr.<x> = RR[]

# ------------------------ PART A START
def proper_divisors_of(n):
  return [i for i in range(1, (n//2) + 1) if n % i == 0 ]

def big_phi(n):
  if (n == 1):
    return x - 1  

  top = (x^n - 1)
  bot = 1
  
  for d in proper_divisors_of(n):
    bot = bot * big_phi(d)

  return top // bot

# TEST FOR PART A:
print("----------------------Q1 PART A -----------------")
print(type(big_phi(9))) # real polynomial
for i in range(1,21):
  print(f"{i=}", cyclotomic_polynomial(i) == big_phi(i))
# ------------------------ PART A END

# ------------------------ PART B START
# ANSWER: The Euler's totient function yields the same degree as the cyclotomic polynomial
print("----------------------Q1 PART B -----------------")
for n in range(1,21):
  print(f"{n=}, cyclotomic polynomial={big_phi(n).degree():},  euler_phi={euler_phi(n)}, EQ={big_phi(n).degree()== euler_phi(n)}")
# ------------------------ PART B END


----------------------Q1 PART A -----------------
<class 'sage.rings.polynomial.polynomial_real_mpfr_dense.PolynomialRealDense'>
i=1 True
i=2 True
i=3 True
i=4 True
i=5 True
i=6 True
i=7 True
i=8 True
i=9 True
i=10 True
i=11 True
i=12 True
i=13 True
i=14 True
i=15 True
i=16 True
i=17 True
i=18 True
i=19 True
i=20 True
----------------------Q1 PART B -----------------
n=1, cyclotomic polynomial=1,  euler_phi=1, EQ=True
n=2, cyclotomic polynomial=1,  euler_phi=1, EQ=True
n=3, cyclotomic polynomial=2,  euler_phi=2, EQ=True
n=4, cyclotomic polynomial=2,  euler_phi=2, EQ=True
n=5, cyclotomic polynomial=4,  euler_phi=4, EQ=True
n=6, cyclotomic polynomial=2,  euler_phi=2, EQ=True
n=7, cyclotomic polynomial=6,  euler_phi=6, EQ=True
n=8, cyclotomic polynomial=4,  euler_phi=4, EQ=True
n=9, cyclotomic polynomial=6,  euler_phi=6, EQ=True
n=10, cyclotomic polynomial=4,  euler_phi=4, EQ=True
n=11, cyclotomic polynomial=10,  euler_phi=10, EQ=True
n=12, cyclotomic polynomial=4,  euler_phi=4, EQ=True
n

In [98]:
# Exercise 2 (9 points)
# Modify the RSA implementation as follows:
# • The modulus (n) is a product of three prime numbers, not two. (What else needs to be changed
# for this to work?) The prime numbers are still random numbers up to 1000.
# • The public exponent (e) is the fixed constant 17 = 24 + 1, and if it doesn’t satisfy the criteria
# of RSA, then generate the prime numbers again, but keep e the same.
# Encrypt the message m = 44800, and then decrypt it back from the ciphertext.
# (Don’t use any operation that would require breaking RSA, e.g. factoring n.)

PUB_EXP = 17

def gen_key():
  p,q,r = random_prime(1000), random_prime(1000), random_prime(1000)
  n = p * q * r
  phi_n = (p - 1) * (q-1) * (r-1)
    
  while phi_n <= PUB_EXP or gcd(PUB_EXP, phi_n) != 1:
    p,q,r = random_prime(1000), random_prime(1000), random_prime(1000)
    n = p * q * r
    phi_n = (p - 1) * (q-1) * (r-1)
  
  var("k")
  r= solve_mod(PUB_EXP * k == 1, phi_n)
  if len(r) == 0:
    return None # Uh oh :(

  d = r[0][0]
  return (n,PUB_EXP), (n, d)


def encrypt(msg, pub_key):
  (n,e) = pub_key
  return power_mod(msg, e, n)

def decrypt(cipher, pri_key):
  (n,d) = pri_key
  return power_mod(cipher, d, n)


pub, pri = gen_key()
ciphertext = encrypt(44800, pub)
text = decrypt(ciphertext, pri)
print("----------------------Q2 -----------------")
print(f"pub_key={pub}, pri_key={pri}")
print(f"ciphertext={ciphertext}")
print(f"decrypt={text}")


----------------------Q2 -----------------
pub_key=(34118027, 17), pri_key=(34118027, 15629553)
ciphertext=27109376
decrypt=44800


In [99]:
# Exercise 3 (9 points = 5 + 4)
# In this exercise, a let R be a commutative ring with identity, given by the list of its elements (in no
# particular order). Don’t assume anything about the elements other than:
# • they have the two ring operations: a + b and a * b,
# • they form a commutative ring with identity,
# • and they can be compared for equality (a == b, a != b).
# In particular, don’t have an assumption about which element is the identity.
# (Exercise continues on the next page.)and an element a ∈ R, and it returns the multiplicative
# inverse of a (i.e. a−1) if it has one, otherwise it returns None.
# Using this function, print the multiplicative inverses of all invertible elements of Z10.
# b) Write a function that gets the ring R, and it returns whether it is a field.
# Using this function, check which of Z1, Z2, . . . , Z20 are fields.
# (Hint: we already had integral domains on the class.)

def get_zero(R):
  for x in R:
    for a in R:
      if a + x != a:
        break # At least one element violate a + x = a
    return x # Every element is a + x == a
  return None

def is_invertible(R):
  z = get_zero(R)
  for i in R:
    if i == z:
      continue
    if get_mul_inverse(R, i) == None:
      return False
  return True


def get_iden(R):
  z = get_zero(R)
  for e in R:
    if e == z:
      continue
    for a in R:
      if a * e != a or a * e != a:
        break # At least one element violate ae = a or ea =a
    return e # Every element is a * e = a
  return None

def get_mul_inverse(R, a):
  e = get_iden(R) # Assumed ring with identity
  for x in R:
    if a * x == e: # Assumed commutative ring
      return x
  return None

# PART A TESTS
print("----------------------Q3 PART A-----------------")
Z10 = Zmod(10)
for z in Z10:
    print(f"{z=} {get_mul_inverse(Z10, z)=}")

# PART B
def is_field(R):
  _c1 = len(R) >= 2
  # Assume R is commutative
  # _c2 = is_commutative(R)
  _c2 = true
  _c3 = get_iden(R) != None
  _c4 = is_invertible(R)
  return _c1 and _c2 and _c3 and _c4

print("----------------------Q3 PART B-----------------")
for n in range(1,21):
  print(f"{n=}, is_field={is_field(Zmod(n))}")


----------------------Q3 PART A-----------------
z=0 get_mul_inverse(Z10, z)=None
z=1 get_mul_inverse(Z10, z)=1
z=2 get_mul_inverse(Z10, z)=None
z=3 get_mul_inverse(Z10, z)=7
z=4 get_mul_inverse(Z10, z)=None
z=5 get_mul_inverse(Z10, z)=None
z=6 get_mul_inverse(Z10, z)=None
z=7 get_mul_inverse(Z10, z)=3
z=8 get_mul_inverse(Z10, z)=None
z=9 get_mul_inverse(Z10, z)=9
----------------------Q3 PART B-----------------
n=1, is_field=False
n=2, is_field=True
n=3, is_field=True
n=4, is_field=False
n=5, is_field=True
n=6, is_field=False
n=7, is_field=True
n=8, is_field=False
n=9, is_field=False
n=10, is_field=False
n=11, is_field=True
n=12, is_field=False
n=13, is_field=True
n=14, is_field=False
n=15, is_field=False
n=16, is_field=False
n=17, is_field=True
n=18, is_field=False
n=19, is_field=True
n=20, is_field=False


In [100]:
# Exercise 4 (8 points)
# Find the smallest number which is a Fermat pseudoprime for all of the bases a = 2, 3, 4, but not a
# Carmichael number.

def is_fermat_prime(n,a):
  if (n < 4):
    return None

  if a < 2 or a > n-2:
    return None

  return power_mod(a, n-1, a) != 1

def is_carmichael(n):
  cnt = 0
  for a in range(1,n):
    if (power_mod(a, n-1, n) != 1):
      cnt += 1
  return cnt >= n // 2

print(f"---------------- Q4-------------")
found = False
n = 4
while not found:
  c1 = is_fermat_prime(n,2)
  c2 = is_fermat_prime(n,3)
  c3 = is_fermat_prime(n,4)
  c4 = not is_carmichael(n)
  print(f"trying {n=}")
  if c1 and c2 and c3 and c4:
    found = True
  else:
      n += 1
  
print(f"Disclaimer: test claims 50% certainty for a carmichael nunber :(")
print(f"Found {n=}")

---------------- Q4-------------
trying n=4
trying n=5
trying n=6
trying n=7
Disclaimer: test claims 50% certainty for a carmichael nunber :(
Found n=7


In [101]:
# Exercise 5 (6 points)
# We know that 1/x is not a polynomial. There is not even such a polynomial that has the same values
# as 1/x for all x̸ = 0 – at least not with numbers. But over finite rings, it is possible. Create the
# polynomial p ∈ Z149[x] of degree 147 for which ∀a ∈ Z∗
# 149 : p(a) = 1/a = a−1.

Z149 = Zmod(149)
P.<x> = Z149[]
z_Z149 = get_zero(Z149) # from prev Q
p_pair = []
for a in Z149:
  if a == z :
    continue
  a_inv = get_mul_inverse(Z149, a) # from prev Q
  if a_inv == None:
    continue

  p_pair.append((a, a_inv))
  

p = P.lagrange_polynomial(p_pair)
print("-----------------------Q5---------------")
print(p)
for a in Z149:
  if a == z:
    continue
  print(f"{a=}, {p(a)=}, 1/a={get_mul_inverse(Z149, a)}")


-----------------------Q5---------------
x^147
a=0, p(a)=0, 1/a=None
a=1, p(a)=1, 1/a=1
a=2, p(a)=75, 1/a=75
a=3, p(a)=50, 1/a=50
a=4, p(a)=112, 1/a=112
a=5, p(a)=30, 1/a=30
a=6, p(a)=25, 1/a=25
a=7, p(a)=64, 1/a=64
a=8, p(a)=56, 1/a=56
a=9, p(a)=116, 1/a=116
a=10, p(a)=15, 1/a=15
a=11, p(a)=122, 1/a=122
a=12, p(a)=87, 1/a=87
a=13, p(a)=23, 1/a=23
a=14, p(a)=32, 1/a=32
a=15, p(a)=10, 1/a=10
a=16, p(a)=28, 1/a=28
a=17, p(a)=114, 1/a=114
a=18, p(a)=58, 1/a=58
a=19, p(a)=102, 1/a=102
a=20, p(a)=82, 1/a=82
a=21, p(a)=71, 1/a=71
a=22, p(a)=61, 1/a=61
a=23, p(a)=13, 1/a=13
a=24, p(a)=118, 1/a=118
a=25, p(a)=6, 1/a=6
a=26, p(a)=86, 1/a=86
a=27, p(a)=138, 1/a=138
a=28, p(a)=16, 1/a=16
a=29, p(a)=36, 1/a=36
a=30, p(a)=5, 1/a=5
a=31, p(a)=125, 1/a=125
a=32, p(a)=14, 1/a=14
a=33, p(a)=140, 1/a=140
a=34, p(a)=57, 1/a=57
a=35, p(a)=132, 1/a=132
a=36, p(a)=29, 1/a=29
a=37, p(a)=145, 1/a=145
a=38, p(a)=51, 1/a=51
a=39, p(a)=107, 1/a=107
a=40, p(a)=41, 1/a=41
a=41, p(a)=40, 1/a=40
a=42, p(a)=110, 1/a=